In [2]:
import importlib
importlib.reload(cdl)

NameError: name &#39;cdl&#39; is not defined

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import time
#from load_dataset import load_mnist_test, load_mnist_train
#from load_dataset import load_fashion_test, load_fashion_train
import load_dataset as data
#from conduct_svm import svm
import Convert as con
import ConvDL as cdl
import conduct_svm as svm
import conduct_pooling as cp
import output as op
import random

In [19]:
train_amount = 48

import scipy.io as sio
mat_contents = sio.loadmat("./YaleB_32x32.mat", appendmat=True)
# 今回はきれいにデータが整理されている31クラスを使用する
img = mat_contents.get("fea")
label = mat_contents.get("gnd")
print(img.shape)
print(label.shape)

u, cnt = np.unique(label, return_counts=True)
id_label = []
prev = 0
for i in cnt:
    id_label.append(prev + i)
    prev += i

ids_used = []
prev = 0
for i in np.array(id_label)[cnt==64]:
    # print(i)
    start = i * 64 - 1
    a = np.arange(i-64, i)
    # print(a)
    ids_used.append(a)
    prev = i

ids_used = np.concatenate(ids_used)

img = img[ids_used].T.reshape(32, 32, 31, 64)
label = label[ids_used].reshape(31, 64)
# print(S.shape)
# print(label.shape)

idx = np.random.choice(64, 64, replace=False)
train_img = img[0:, 0:, 0:, idx[:train_amount]].reshape(32, 32, -1)
test_img = img[0:, 0:, 0:, idx[train_amount:]].reshape(32, 32, -1)
train_label = label[0:, idx[:train_amount]].reshape(-1)
test_label = label[0:, idx[train_amount:]].reshape(-1)

(2414, 1024)
(2414, 1)


In [8]:
print(train_img.shape)
print(train_label.shape)

(32, 32, 1488)
(1488,)


In [20]:
def subtract_mean(S):
    return S - np.mean(S, axis=(0, 1))

from mysporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load, gpu_info)
from mysporco.cupy.dictlrn import onlinecdl
from mysporco.cupy.admm import cbpdn

if not cupy_enabled():
    print('CuPy/GPU device not available: running without GPU acceleration\n')
else:
    id = select_device_by_load()
    info = gpu_info()
    if info:
        print('Running on GPU %d (%s)\n' % (id, info[id].name))

# 辞書の初期化
np.random.seed(12345)
D0 = np.random.randn(9, 9, 9)
dsz = ((3, 3, 3),(6, 6, 3),(9 , 9, 3))

# 辞書学習の実行(ライブラリ使用)
lmbda = 0.05
S = subtract_mean(train_img)
opt = onlinecdl.OnlineConvBPDNDictLearn.Options({
                'Verbose': True, 'DictSize': dsz, 'ZeroMean': True, 'eta_a': 10.0,
                 'eta_b': 20.0, 'DataType': np.float32,
                'CBPDN': {'rho': 50.0*lmbda + 0.5, 'NonNegCoef': True, 'RelStopTol': 1e-3, 
                          'HighMemSolve': True, 'FastSolve': True, 'DataType': np.float32}})
                          
d = onlinecdl.OnlineConvBPDNDictLearn(np2cp(D0), lmbda, opt)

d.display_start()
while True:
  indexes = np.array([i for i in range(train_img.shape[-1])])
  np.random.shuffle(indexes)
  for i in indexes:
    a = np2cp(S[0:, 0:, i])
    d.solve(a)
    if (d.getitstat().DeltaD[-1] < 1e-5):  break
  else:
      continue
d.display_end()

D1 = cp2np(d.getdict())

Running on GPU 0 (GeForce RTX 2070)

(9, 9, 9)
1
Itn   X r       X s       X ρ       D cnstr   D dlt     D η     
----------------------------------------------------------------
float32
float32


TypeError: Unsupported type <class 'numpy.ndarray'>

In [14]:
S

dtype('float64')

In [4]:
def subtract_mean(S):
    return S - np.mean(S, axis=(0, 1))

from mysporco.dictlrn import cbpdndl

# 辞書の初期化
np.random.seed(1)
D0 = np.random.randn(11, 11, 6)
dsz = D0.shape
dsz = ((4, 4, 3), (11, 11, 3))
# cri = cr.CSC_ConvRepIndexing(D0, S)

# 設定
lmbda = 0.5
opt = cbpdndl.ConvBPDNDictLearn.Options({'Verbose': True, 'MaxMainIter': 1000, 'DictSize': dsz,
                            'CBPDN': {'rho': 50.0*lmbda + 0.5, 'NonNegCoef': True, 'RelStopTol': 1e-3, 'HighMemSolve': True, 'FastSolve': False},
                            'CCMOD': {'ZeroMean': True}},
                            dmethod='cns')
S = subtract_mean(train_img)
d = cbpdndl.ConvBPDNDictLearn(D0, S, lmbda, opt)

D1 = d.solve()

(11, 11, 6)
1
Itn   Fnc       DFid      ℓ1        Cnstr     r_X       s_X       ρ_X       r_D       s_D       ρ_D     
--------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [8]:
D1 = d.getdict()
X1 = d.getcoef().squeeze()
print(D1.shape)
print(X1.shape)

(11, 11, 1, 1, 6)
(32, 32, 1488, 6)


In [10]:
import mycbpdndl

# 辞書の初期化
np.random.seed(1)
D0 = np.random.randn(11, 11, 6, 10)
dsz = ((4, 4, 6, 5), (11, 11, 6, 5))
# cri = cr.CSC_ConvRepIndexing(D0, S)

# 設定
lmbda = 0.5
opt = mycbpdndl.ConvBPDNDictLearn.Options({'Verbose': True, 'MaxMainIter': 1000, 'DictSize': dsz,
                            'CBPDN': {'rho': 50.0*lmbda + 0.5, 'NonNegCoef': True, 'RelStopTol': 1e-3, 'HighMemSolve': True, 'FastSolve': False},
                            'CCMOD': {'ZeroMean': True}},
                            dmethod='cns')
S = subtract_mean(train_img)
d = mycbpdndl.ConvBPDNDictLearn(D0, X1.transpose(0, 1, 3, 2), lmbda, opt)

D2 = d.solve()

(11, 11, 6, 10)
6


KeyboardInterrupt: 

In [12]:
D2 = d.getdict()
X2 = d.getcoef().squeeze()
print(D2.shape)
print(X2.shape)

(11, 11, 6, 1, 10)
(32, 32, 6, 1488, 10)


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import time
#from load_dataset import load_mnist_test, load_mnist_train
#from load_dataset import load_fashion_test, load_fashion_train
import load_dataset as data
#from conduct_svm import svm
import Convert as con
import ConvDL as cdl
import conduct_svm as svm
import conduct_pooling as cp
import output as op
import random


#実験のオプションを設定
def powerspectrum(A):
    
    return np.abs(np.fft.fftshift(np.fft.fft2(A)))


def make_option():
    opt = {}
    # 訓練画像の枚数
    opt['train_amount'] = [200]
    #opt['train_amount'] = [50, 100]
    # テスト画像の枚数
    opt['test_amount'] = 500
    # 最大繰り返し回数
    opt['iteration'] = [1, 100, 3]
    # 更新終了の閾値
    opt['thr'] = 0.001
    #学習の層の数
    opt['Learning'] = 2
    #フィルターのサイズ
    opt['d1_size, d2_size'] = [4, 14]
    #フィルターの数
    opt['d_amount'] = [4, 4]
    #パラメータ設定
    opt['rho_coef'] = 0.2
    opt['rho_dic_conventional'] = 100.0 #係数の射影
    opt['rho_dic_propposed'] = 0.1 #係数の射影
    opt['mu'] = 1e-05 #勾配法
    opt['gamma_conventional'] = 1.0  #近似誤差(L1)
    opt['gamma_propposed'] = 1.0 #近似誤差(L1)
    #実験結果の保存先
    opt['file_path'] = 'C:\\CDL\\result'
    return opt
print("start")
opt = make_option() 

roop = [1]
    
    
for seed in range(20,100):
            
    #辞書の初期値設定
    def dictionary_set(learning, d_size, d_amount):
        D = []
        for i in range(learning):
            np.random.seed(seed=seed)
            D.append(np.random.normal(0, 1, (d_size[i], d_size[i], d_amount[i])))
        return D


    Learned_D1 = []
    Learned_D2 = []

    D0 = dictionary_set(opt['Learning'], opt['d1_size, d2_size'], opt['d_amount'])
    op.output_Image(0, D0, 'Initial_Dictionary', opt['file_path'])

    print(D0[0].shape)
    print(opt)

    fname = 'result_conventional.txt'
    #fname = 'result_proposed.txt'

    for train_amount in opt['train_amount']:
        #opt['rho_coef'] = train_amount * 0.001

        #train_data, train_label = load_mnist_train(train_amount)
        #test_data, test_label = load_mnist_test(opt["test_amount"])

        train_data, train_label = data.load_fashion_train(train_amount)
        test_data, test_label = data.load_fashion_test(opt["test_amount"])

        '''for i in range(int(train_amount/100)):
            outlier = random.randrange(0, train_amount, 1)
            train_data[outlier] = 0'''

        train_data = train_data.transpose(1,2,0)
        test_data = test_data.transpose(1,2,0)
        op.output_Image(3, train_data, 'Fashion-MNIST', opt['file_path'])

        print("\nTrain data: " + str(train_amount) + "\n")

        time_start = time.time()

        #print("\n[gamma: %1.2f, rho_dic: %1.2f, mu: %1.2f]" % (opt['gamma_conventional'], opt['rho_dic_conventional'], opt['mu']))
        #print("\n[rho_coef: %1.2f, rho_dic: %1.2f, mu: %1.2f, gamma: %1.2f]" % (opt['rho_coef'], opt['rho_dic_propposed'], opt['mu'], opt['gamma']))

        #D1, X1, L0_1 = cdl.coef_dic_update_L2_L0(opt['gamma_conventional'], opt['rho_dic_conventional'], opt['mu'], train_data, D0[0], opt['iteration'], opt['thr'], 'Layer1')            
        D1, X1, L0_1 = cdl.coef_dic_update_L1_L0(opt['rho_coef'], opt['rho_dic_propposed'], opt['mu'], opt['gamma_propposed'], train_data, D0[0], opt['iteration'], opt['thr'], 'Layer1')
        if(L0_1 == 0):
            continue
        
        X1 = X1.transpose(2,3,0,1)
        Xp1 = cp.pooling_layer(X1, train_amount,32)
        Xe1 = np.zeros((train_amount,14,14))
        for i in range(train_amount):
            Xe1[i,:,:] = np.abs(np.fft.fftshift(np.fft.fft2(Xp1[i,:,:])))[6:20,6:20]
        Xe1 = Xe1.transpose(1, 2, 0)
        
        

        #D2, X2, L0_2 = cdl.coef_dic_update_L2_L0(opt['gamma_conventional'], opt['rho_dic_conventional'], opt['mu'], Xp1, D0[1], opt['iteration'], opt['thr'], 'Layer2')
        D2, X2, L0_2 = cdl.coef_dic_update_L1_L0(opt['rho_coef'], opt['rho_dic_propposed'], opt['mu'], opt['gamma_propposed'], Xe1, D0[1], opt['iteration'], opt['thr'], 'Layer2')
        if(L0_2 == 0):
            continue

        #np.save("train_amount:"+str(train_amount)+" "+str(seed)+"_D1",D1)
        #np.save("train_amount:"+str(train_amount)+" "+str(seed)+"_D2",D2)

        #train_F1, L0_train1 = cdl.feature_extraction_L2_L0(opt['gamma_conventional'], opt['mu'], train_data, D1, opt['iteration'], opt['thr'], 'Feature1')
        train_F1, L0_train1 = cdl.feature_extraction_L1_L0(opt['rho_coef'], opt['gamma_propposed'], opt['mu'], train_data, D1, opt['iteration'], opt['thr'], 'Feature1')
        if(L0_train1 == 0):
            continue
        train_F1 = train_F1.transpose(2,3,0,1)
        train_F1_P = cp.pooling_layer(train_F1, train_amount,32)
        train_F1_E = np.zeros((train_amount,14,14))
        for i in range(train_amount):
            train_F1_E[i,:,:] = np.abs(np.fft.fftshift(np.fft.fft2(train_F1_P[i,:,:])))[6:20,6:20]
        train_F1_E = train_F1_E.transpose(1, 2, 0)

        #train_F2, L0_train2 = cdl.feature_extraction_L2_L0(opt['gamma_conventional'], opt['mu'], train_F1_P, D2, opt['iteration'], opt['thr'], 'Feature2')
        train_F2, L0_train2 = cdl.feature_extraction_L1_L0(opt['rho_coef'], opt['gamma_propposed'], opt['mu'], train_F1_E, D2, opt['iteration'], opt['thr'], 'Feature2')
        if(L0_train2 == 0):
            continue

        train_F2 = train_F2.transpose(2, 3, 0, 1)
        train_F2_P = cp.pooling_layer(train_F2, train_amount,14)
        train_F2_E = np.zeros((train_amount,4,4))
        for i in range(train_amount):
            train_F2_E[i,:,:] = np.abs(np.fft.fftshift(np.fft.fft2(train_F2_P[i,:,:])))[5:9,5:9]
        
        train_F2_E = train_F2_E.reshape(train_amount,-1)
        #train_F2 = train_F2.transpose(0,1,3,2)
        #train_F2 = train_F2.reshape(-1,train_F2.shape[3])

        clf = svm.train_svm(train_F2_E, train_label)

        #test_F1, L0_test1 = cdl.feature_extraction_L2_L0(opt['gamma_conventional'], opt['mu'], test_data, D1, opt['iteration'], opt['thr'], 'Feature1')
        test_F1, L0_test1 = cdl.feature_extraction_L1_L0(opt['rho_coef'], opt['gamma_propposed'], opt['mu'], test_data, D1, opt['iteration'], opt['thr'], 'Feature1')
        if(L0_test1 == 0):
            continue

        test_F1 = test_F1.transpose(2, 3, 0, 1)
        test_F1_P = cp.pooling_layer(test_F1, opt['test_amount'],32)
        test_F1_E = np.zeros((opt['test_amount'],14,14))
        for i in range(opt['test_amount']):
             test_F1_E[i,:,:] = np.abs(np.fft.fftshift(np.fft.fft2(test_F1_P[i,:,:])))[6:20,6:20]
        test_F1_E = test_F1_E.transpose(1, 2, 0)

        #test_F2, L0_test2 = cdl.feature_extraction_L2_L0(opt['rho_coef'], opt['mu'], test_F1_P, D2, opt['iteration'], opt['thr'], 'Feature2')
        test_F2, L0_test2 = cdl.feature_extraction_L1_L0(opt['rho_coef'], opt['gamma_propposed'], opt['mu'], test_F1_E, D2, opt['iteration'], opt['thr'], 'Feature2')
        if(L0_test2 == 0):
            continue

        test_F2 = test_F2.transpose(2, 3, 0, 1)
        test_F2_P = cp.pooling_layer(test_F2, opt['test_amount'],14)
        test_F2_E = np.zeros((opt['test_amount'],4,4))
        for i in range(opt['test_amount']):
            test_F2_E[i,:,:] = np.abs(np.fft.fftshift(np.fft.fft2(test_F2_P[i,:,:])))[5:9,5:9]
        test_F2_E = test_F2_E.reshape(opt['test_amount'],-1)

        #test_F2 = test_F2.transpose(0,1,3,2)
        #test_F2 = test_F2.reshape(-1,test_F2.shape[3])

        accuracy = svm.test_svm(clf, test_F2_E, test_label)

        time_end = time.time()

        print("accuracy: %1.3f" % accuracy)
        print("solve time: %.2fs\n" % (time_end-time_start))

        Learned_D1.append(D1)
        Learned_D2.append(D2)

        #op.output_Text_conventional(fname, train_amount, accuracy, opt['d_amount'], opt['gamma_conventional'], opt['rho_dic_conventional'], opt['mu'], train_F1.size, train_F2.size, test_F1.size, test_F2.size, L0_train1, L0_train2, L0_test1, L0_test2, opt['file_path'])
        #op.output_Text_proposed(fname, train_amount, accuracy, opt['d_amount'], opt['rho_coef'], opt['rho_dic_propposed'], opt['mu'], opt['gamma'], train_F1.size, train_F2.size, test_F1.size, test_F2.size, L0_train1, L0_train2, L0_test1, L0_test2, opt['file_path'])

        with open("result.txt",mode = 'a')  as  f:
            f.write("\n")
            f.write("制約なし　プーリング＋パワースペクトル(2)+cut high frewuency14+4(2)\n")
            f.write(str(opt)+"\n")
            f.write(str(train_amount)+"　"+str(seed)+"\n")
            f.write("score："+str(accuracy)+"\n")
            f.write("\n")

    #op.output_Image(1, Learned_D1, 'Learned_Dictionary_Conventional(Layer1)', opt['file_path'])
    #op.output_Image(2, Learned_D2, 'Learned_Dictionary_Conventional(Layer2)', opt['file_path'])
    #op.output_Image(1, Learned_D1, 'Learned_Dictionary_Proposed(Layer1)', opt['file_path'])
    #op.output_Image(2, Learned_D2, 'Learned_Dictionary_Proposed(Layer2)', opt['file_path'])
    Learned_D1.clear()
    Learned_D2.clear()




ModuleNotFoundError: No module named &#39;cv2.cv2&#39;